In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable
import os
import pickle
from tqdm import notebook, tqdm

In [3]:
with open('data/train_128.pickle', 'rb') as train_pickle:
    train_data = pickle.load(train_pickle)
    
with open('data/valid_128.pickle', 'rb') as valid_pickle:
    valid_data = pickle.load(valid_pickle)

with open('data/test_128.pickle', 'rb') as test_pickle:
    test_data = pickle.load(test_pickle)

In [4]:
class MultiResBlock(nn.Module):
    def __init__(self,ch_in,ch_out):
        super().__init__()
        self.conv1x1 = nn.Conv2d(ch_in, ch_out, kernel_size=1, padding=0)
        self.bnorm1x1 = nn.BatchNorm2d(ch_out,track_running_stats=False)
        self.fconv = nn.Conv2d(ch_in, ch_out//6, kernel_size=3, padding=1)
        self.fbnorm = nn.BatchNorm2d(ch_out//6,track_running_stats=False)
        self.sconv = nn.Conv2d(ch_out//6, ch_out//3, kernel_size=3, padding=1)
        self.sbnorm = nn.BatchNorm2d(ch_out//3,track_running_stats=False)
        self.tconv = nn.Conv2d(ch_out//3, ch_out//2+1, kernel_size=3, padding=1)
        self.tbnorm = nn.BatchNorm2d(ch_out//2+1,track_running_stats=False)
        self.relu = nn.ReLU()

    def forward(self,x):
        res1x1 = self.relu(self.bnorm1x1(self.conv1x1(x)))
        #print("res1x1 done")
        first = self.relu(self.fbnorm(self.fconv(x)))
        #print("fconv done")
        second = self.relu(self.sbnorm(self.sconv(first)))
        third = self.relu(self.tbnorm(self.tconv(second)))
        resconv = torch.cat((first,second,third),dim=1)
        y = res1x1+resconv
        return y

In [5]:
'''class Generator_Res(nn.Module):
    def __init__(self):
        super().__init__()
        self.tr1 = MultiResBlock(512, 256) 
        self.tr2 = MultiResBlock(256, 128)
        self.tr3 = MultiResBlock(128, 64)
        self.tr4 = MultiResBlock(64, 32)
        self.tr5 = MultiResBlock(32, 16)
        self.tr6 = MultiResBlock(16, 8)
        self.tr7 = nn.Conv2d(8, 1, kernel_size = 3, stride = 1, padding = 1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(1024, 512*4*4)
        self.sigmoid = nn.Sigmoid()
        self.upsample = nn.Upsample(scale_factor = 2)
        
        self.up1 = nn.ConvTranspose2d(256, 256, kernel_size = 4, stride = 2, padding = 1)
        self.up2 = nn.ConvTranspose2d(128, 128, kernel_size = 4, stride = 2, padding = 1)
        self.up3 = nn.ConvTranspose2d(64, 64, kernel_size = 4, stride = 2, padding = 1)
        self.up4 = nn.ConvTranspose2d(32, 32, kernel_size = 4, stride = 2, padding = 1)
        self.up5 = nn.ConvTranspose2d(16, 16, kernel_size = 4, stride = 2, padding = 1)
        


    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 512, 4, 4)
        z = self.tr1(z)
        z = self.up1(z) # 8x8
        z = self.tr2(z)
        z = self.up2(z) # 16x16
        z = self.tr3(z)
        z = self.up3(z) # 32x32
        z = self.tr4(z)
        z = self.up4(z) # 64x64
        z = self.tr5(z)
        z = self.up5(z) # 128x128
        z = self.tr6(z)
        z = self.tr7(z)
        z = self.sigmoid(z)
        return z'''


'class Generator_Res(nn.Module):\n    def __init__(self):\n        super().__init__()\n        self.tr1 = MultiResBlock(512, 256) \n        self.tr2 = MultiResBlock(256, 128)\n        self.tr3 = MultiResBlock(128, 64)\n        self.tr4 = MultiResBlock(64, 32)\n        self.tr5 = MultiResBlock(32, 16)\n        self.tr6 = MultiResBlock(16, 8)\n        self.tr7 = nn.Conv2d(8, 1, kernel_size = 3, stride = 1, padding = 1)\n        self.relu = nn.ReLU()\n        self.fc = nn.Linear(1024, 512*4*4)\n        self.sigmoid = nn.Sigmoid()\n        self.upsample = nn.Upsample(scale_factor = 2)\n        \n        self.up1 = nn.ConvTranspose2d(256, 256, kernel_size = 4, stride = 2, padding = 1)\n        self.up2 = nn.ConvTranspose2d(128, 128, kernel_size = 4, stride = 2, padding = 1)\n        self.up3 = nn.ConvTranspose2d(64, 64, kernel_size = 4, stride = 2, padding = 1)\n        self.up4 = nn.ConvTranspose2d(32, 32, kernel_size = 4, stride = 2, padding = 1)\n        self.up5 = nn.ConvTranspose2d(16,

In [6]:
class Generator_Res2(nn.Module):
    def __init__(self):
        super().__init__()
        self.tr1 = MultiResBlock(256, 128)
        self.tr2 = MultiResBlock(128, 64)
        self.tr3 = MultiResBlock(64, 32)
        self.tr4 = MultiResBlock(32, 16)
        self.tr5 = MultiResBlock(16, 8)
        self.tr6 = nn.Conv2d(8, 1, kernel_size = 3, stride = 1, padding = 1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(1024, 256*16*16)
        self.sigmoid = nn.Sigmoid()
        self.upsample = nn.Upsample(scale_factor = 2)

    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 256, 16, 16)
        z = self.tr1(z)
        z = self.upsample(z)
        z = self.tr2(z)
        z = self.upsample(z)
        z = self.tr3(z)
        z = self.upsample(z)
        z = self.tr4(z)
        z = self.tr5(z)
        z = self.tr6(z)
        z = self.sigmoid(z)
        return z

In [7]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [8]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        '''
        Conv2d: w' = (w - k +2p)/s + 1
        ConvTr2d: w = s(w'-1) + k -2p
        '''
        #[b, 512, 4, 4]
        self.tr1 = nn.ConvTranspose2d(512, 384, kernel_size = 4, stride = 2, padding = 1) # [b, 384, 8, 8]
        self.tr2 = nn.ConvTranspose2d(384, 256, kernel_size = 3, stride = 1, padding = 1) # [b, 256, 8, 8]
        self.tr3 = nn.ConvTranspose2d(256, 192, kernel_size = 4, padding = 1, stride = 2) # [b, 192, 16, 16]
        self.tr4 = nn.ConvTranspose2d(192, 128, kernel_size = 3, stride = 1, padding = 1) # [b, 128, 16, 16]
        self.tr5 = nn.ConvTranspose2d(128, 96, kernel_size = 4, padding = 1, stride = 2) # [b, 96, 32, 32]
        self.tr6 = nn.ConvTranspose2d(96, 64, kernel_size = 3, stride = 1, padding = 1) # [b, 64, 32, 32]
        self.tr7 = nn.ConvTranspose2d(64, 48, kernel_size = 4, padding = 1, stride = 2) # [b, 48, 32, 32]
        self.tr8 = nn.ConvTranspose2d(48, 32, kernel_size = 3, stride = 1, padding = 1) # [b, 32, 64, 64]
        self.tr9 = nn.ConvTranspose2d(32, 16, kernel_size = 4, padding = 1, stride = 2) # [b, 16, 64, 64]
        self.tr10 = nn.ConvTranspose2d(16, 8, kernel_size = 3, stride = 1, padding = 1) # [b, 8, 128, 128]
        self.tr11 = nn.ConvTranspose2d(8, 4, kernel_size = 3, padding = 1, stride = 1) # [b, 4, 128, 128]
        self.tr12 = nn.ConvTranspose2d(4, 1, kernel_size = 3, stride = 1, padding = 1) # [b, 1, 128, 128]
        
        
        self.bn1 = nn.BatchNorm2d(384)
        self.bn2 = nn.BatchNorm2d(256)
        self.bn3 = nn.BatchNorm2d(192)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(96)
        self.bn6 = nn.BatchNorm2d(64)
        self.bn7 = nn.BatchNorm2d(48)
        self.bn8 = nn.BatchNorm2d(32)
        self.bn9 = nn.BatchNorm2d(16)
        self.bn10 = nn.BatchNorm2d(8)
        self.bn11 = nn.BatchNorm2d(4)
        
        '''
        self.tr1 = nn.ConvTranspose2d(256, 128, kernel_size = 4, stride = 2, padding = 1)
        self.tr2 = nn.ConvTranspose2d(128, 64, kernel_size = 4, stride = 2, padding = 1)
        self.tr3 = nn.ConvTranspose2d(64, 32, kernel_size = 4, stride = 2, padding = 1)
        self.tr4 = nn.ConvTranspose2d(32, 16, kernel_size = 3, stride = 1, padding = 1)
        self.tr5 = nn.ConvTranspose2d(16, 8, kernel_size = 3, stride = 1, padding = 1)
        self.tr6 = nn.ConvTranspose2d(8, 1, kernel_size = 3, stride = 1, padding = 1)'''
        self.relu = nn.ReLU()
        self.fc = nn.Linear(1024, 512*4*4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, z):
        z = self.fc(z)
        z = z.view(-1, 512, 4, 4)
        z1 = self.bn1(self.relu(self.tr1(z)))
        # z = [b, 512, 4, 4], z1 = [b, 384, 8, 8], z2 [b, 256, 8, 8] -> [b, 640, 8, 8]
        z2 = self.bn2(self.relu(self.tr2(z1)))
        z3 = self.bn3(self.relu(self.tr3(z2)))
        z4 = self.bn4(self.relu(self.tr4(z3)))
        z5 = self.bn5(self.relu(self.tr5(z4)))
        z6 = self.bn6(self.relu(self.tr6(z5)))
        z7 = self.bn7(self.relu(self.tr7(z6)))
        z8 = self.bn8(self.relu(self.tr8(z7)))
        z9 = self.bn9(self.relu(self.tr9(z8)))
        z10 = self.bn10(self.relu(self.tr10(z9)))
        z11 = self.bn11(self.relu(self.tr11(z10)))
        z12 = self.sigmoid(self.tr12(z11))
        return z12


In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size = 7, stride =2)#, padding = 1)
        self.conv2 = nn.Conv2d(4, 8, kernel_size = 5, stride = 2)#, padding = 1)
        self.conv3 = nn.Conv2d(8, 16, kernel_size = 3, stride = 1, padding = 1)
        self.fc1 = nn.Linear(16*29*29, 1)
        self.maxpool = nn.MaxPool2d(2)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        #x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        #x = self.maxpool(x)
        x = self.conv3(x)
        x = self.relu(x)
        #x = self.maxpool(x)
        x = x.view(-1, 29*29*16)
        logit = self.fc1(x)
        #logit = self.fc2(x)
        x = self.sigmoid(logit)
        return x, logit

In [10]:
class Discriminator_2(nn.Module):
    '''
    Conv2d: w' = (w - k +2p)/s + 1
    ConvTr2d: w = s(w'-1) + k -2p
    '''    
    def __init__(self, in_size = 1, ndf = 64):
        super().__init__()       
        self.in_size = in_size
        self.ndf = ndf
        self.main = nn.Sequential(
            # input size is in_size x 128 x 128
            nn.Conv2d(in_size, self.ndf, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: ndf x 64 x 64
            nn.Conv2d(self.ndf, self.ndf, kernel_size = 4, stride = 2, padding = 1, bias = False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: ndf x 32 x 32
            nn.Conv2d(self.ndf, self.ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: (ndf * 2) x 16 x 16
            nn.Conv2d(self.ndf * 2, self.ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: (ndf * 4) x 8 x 8
            nn.Conv2d(self.ndf * 4, self.ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size: (ndf * 8) x 4 x 4
            nn.Conv2d(self.ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            # state size: 1 x 1 x 1
        )
        
        
        
    def forward(self, x):
        output = self.main(x)
        return output

In [11]:
def reduce_mean(x):
    output = torch.mean(x,0, keepdim = False)
    output = torch.mean(output,-1, keepdim = False)
    return output

In [12]:
def train(discriminator, generator, optimizerD, optimizerG, d_criterion, g_criterion, data, batch_size, epoch, device):
    generator.train()
    discriminator.train()
    lossD = []
    lossG = []
    sum_D_x = 0
    sum_loss_D = 0
    sum_loss_G = 0
    it = 0
    for X in batch_generator(data, batch_size, shuffle=True):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        #train with real
        X = torch.from_numpy(X).float().to(device)
        X = X/127
        X = torch.unsqueeze(X,1)
        discriminator.zero_grad()
        
        D, D_logits = discriminator(X)
        
        

        #####loss
        d_loss_real = reduce_mean(d_criterion(D_logits, 0.9*torch.ones_like(D)))
        d_loss_real.backward()#retain_graph=True)
        D_x = D.mean().item()
        sum_D_x += D_x 
        
        
        #train with fake
        
        noise = torch.randn(batch_size, 1024, device=device)
        

        X = generator(noise)
        D_, D_logits_ = discriminator(X)
        d_loss_fake = reduce_mean(d_criterion(D_logits_, torch.zeros_like(D_)))
        d_loss_fake.backward(retain_graph=True)
        
        D_G_z1 = D_.mean().item()
        
        errD = d_loss_fake + d_loss_real
        errD = errD.item()
        
        sum_loss_D += errD
        
        optimizerD.step()
        
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        generator.zero_grad()
                
        D_, D_logits_ = discriminator(X)

        ###loss
        errG = reduce_mean(g_criterion(D_logits_, torch.ones_like(D_)))
        
        errG.backward()#retain_graph=True)
        D_G_z2 = D_.mean().item()
        optimizerG.step()
        
        sum_loss_G += errG
        it += 1
        if it % 100 == 0:
            print(f'Train | D loss={errD:.6f} | G loss:{errG:.6f} | D(x):{D_x:.6f} | D(G(z)) = {D_G_z1:.4f} / {D_G_z2:.4f}')
    print(f'Train | epoch={epoch:03d} | D loss={sum_loss_D:.6f} | G loss:{sum_loss_G:.6f}')
    


def validate(discriminator, generator, d_criterion, g_criterion, data, batch_size, epoch, device):
    discriminator.eval()
    generator.eval()
    lossD = []
    lossG = []
    sum_D_x = 0
    sum_loss_D = 0
    sum_loss_G = 0
    it = 0
    with torch.no_grad():
        for X in batch_generator(data, batch_size, shuffle=True):
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            #train with real
            X = torch.from_numpy(X).float().to(device)
            X = X/127
            X = torch.unsqueeze(X,1)

            D, D_logits = discriminator(X)



            #####loss
            d_loss_real = reduce_mean(d_criterion(D_logits, 0.9*torch.ones_like(D)))
            D_x = D.mean().item()
            sum_D_x += D_x 


            #train with fake

            noise = torch.randn(batch_size, 1024, device=device)


            X = generator(noise)
            D_, D_logits_ = discriminator(X)
            d_loss_fake = reduce_mean(d_criterion(D_logits_, torch.zeros_like(D_)))

            D_G_z1 = D_.mean().item()

            errD = d_loss_fake + d_loss_real
            errD = errD.item()

            sum_loss_D += errD

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            D_, D_logits_ = discriminator(X)

            ###loss
            errG = reduce_mean(g_criterion(D_logits_, torch.ones_like(D_)))
            D_G_z2 = D_.mean().item()
            sum_loss_G += errG
            it += 1
            if it % 100 == 0:
                print(f'Validation | D loss={errD:.6f} | G loss:{errG:.6f} | D(x):{D_x:.6f} | D(G(z)) = {D_G_z1:.4f} / {D_G_z2:.4f}')
        print(f'Validation | epoch={epoch:03d} | D loss={sum_loss_D:.6f} | G loss:{sum_loss_G:.6f}')

In [13]:
def train2(discriminator, generator, optimizerD, optimizerG, d_criterion, g_criterion, data, batch_size, epoch, device):
    generator.train()
    discriminator.train()
    lossD = []
    lossG = []
    sum_D_x = 0
    sum_loss_D = 0
    sum_loss_G = 0
    it = 0
    for X in batch_generator(data, batch_size, shuffle=True):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        #train with real
        X = torch.from_numpy(X).float().to(device)
        X = X/127
        X = torch.unsqueeze(X,1)
        discriminator.zero_grad()
        
        D = discriminator(X).view(-1)
        
        

        #####loss
        d_loss_real = reduce_mean(d_criterion(D, 0.9*torch.ones_like(D)))
        d_loss_real.backward()#retain_graph=True)
        D_x = D.mean().item()
        sum_D_x += D_x 
        
        
        #train with fake
        
        noise = torch.randn(batch_size, 1024, device=device)
        

        X = generator(noise)
        D_ = discriminator(X).view(-1)
        d_loss_fake = reduce_mean(d_criterion(D_, torch.zeros_like(D_)))
        d_loss_fake.backward(retain_graph=True)
        
        D_G_z1 = D_.mean().item()
        
        errD = d_loss_fake + d_loss_real
        errD = errD.item()
        
        sum_loss_D += errD
        
        optimizerD.step()
        
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        generator.zero_grad()
                
        D_ = discriminator(X).view(-1)

        ###loss
        errG = reduce_mean(g_criterion(D_, torch.ones_like(D_)))
        
        errG.backward()#retain_graph=True)
        D_G_z2 = D_.mean().item()
        optimizerG.step()
        
        sum_loss_G += errG
        it += 1
        if it % 100 == 0:
            print(f'Train | D loss={errD:.6f} | G loss:{errG:.6f} | D(x):{D_x:.6f} | D(G(z)) = {D_G_z1:.4f} / {D_G_z2:.4f}')
    print(f'Train | epoch={epoch:03d} | D loss={sum_loss_D:.6f} | G loss:{sum_loss_G:.6f}')
    


def validate2(discriminator, generator, d_criterion, g_criterion, data, batch_size, epoch, device):
    discriminator.eval()
    generator.eval()
    lossD = []
    lossG = []
    sum_D_x = 0
    sum_loss_D = 0
    sum_loss_G = 0
    it = 0
    with torch.no_grad():
        for X in batch_generator(data, batch_size, shuffle=True):
            ############################
            # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
            ###########################
            #train with real
            X = torch.from_numpy(X).float().to(device)
            X = X/127
            X = torch.unsqueeze(X,1)

            D = discriminator(X).view(-1)



            #####loss
            d_loss_real = reduce_mean(d_criterion(D, 0.9*torch.ones_like(D)))
            D_x = D.mean().item()
            sum_D_x += D_x 


            #train with fake

            noise = torch.randn(batch_size, 1024, device=device)


            X = generator(noise)
            D_ = discriminator(X).view(-1)
            d_loss_fake = reduce_mean(d_criterion(D_, torch.zeros_like(D_)))

            D_G_z1 = D_.mean().item()

            errD = d_loss_fake + d_loss_real
            errD = errD.item()

            sum_loss_D += errD

            ############################
            # (2) Update G network: maximize log(D(G(z)))
            ###########################
            D_ = discriminator(X).view(-1)

            ###loss
            errG = reduce_mean(g_criterion(D_, torch.ones_like(D_)))
            D_G_z2 = D_.mean().item()
            sum_loss_G += errG
            it += 1
            if it % 100 == 0:
                print(f'Validation | D loss={errD:.6f} | G loss:{errG:.6f} | D(x):{D_x:.6f} | D(G(z)) = {D_G_z1:.4f} / {D_G_z2:.4f}')
        print(f'Validation | epoch={epoch:03d} | D loss={sum_loss_D:.6f} | G loss:{sum_loss_G:.6f}')

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
epochs = 20
batch_size = 64

In [16]:
def batch_generator(data, batch_size=1, shuffle=False):
    nsamples = len(data)
    if shuffle:
        perm = np.random.permutation(nsamples)
    else:
        perm = range(nsamples)

    for i in range(0, nsamples, batch_size):
        batch_idx = perm[i:i+batch_size]
        yield data[batch_idx]

In [17]:
d = Discriminator().to(device)
g = Generator().to(device)

d.apply(weights_init)
g.apply(weights_init)

optimizerD = torch.optim.Adam(d.parameters(), lr=0.0004, betas=(0.5, 0.999))
optimizerG = torch.optim.Adam(g.parameters(), lr=0.001, betas=(0.5, 0.999))

d_criterion = nn.BCEWithLogitsLoss()
g_criterion = nn.BCEWithLogitsLoss()

for epoch in range(epochs):
    train(d, g, optimizerD, optimizerG, d_criterion, g_criterion, train_data, batch_size=64, epoch=epoch, device=device)
    val_loss = validate(d, g, d_criterion, g_criterion, train_data, batch_size=64, epoch=epoch, device=device)


s:3.466489 | D(x):0.836530 | D(G(z)) = 0.0312 / 0.0312
Validation | D loss=0.538645 | G loss:3.468076 | D(x):0.829236 | D(G(z)) = 0.0312 / 0.0312
Validation | D loss=0.447932 | G loss:3.470951 | D(x):0.860061 | D(G(z)) = 0.0311 / 0.0311
Validation | D loss=0.492305 | G loss:3.465893 | D(x):0.840629 | D(G(z)) = 0.0312 / 0.0312
Validation | D loss=0.419216 | G loss:3.469617 | D(x):0.875769 | D(G(z)) = 0.0311 / 0.0311
Validation | D loss=0.464862 | G loss:3.465104 | D(x):0.860023 | D(G(z)) = 0.0313 / 0.0313
Validation | D loss=0.449150 | G loss:3.465120 | D(x):0.856109 | D(G(z)) = 0.0313 / 0.0313
Validation | D loss=0.410586 | G loss:3.467199 | D(x):0.872097 | D(G(z)) = 0.0312 / 0.0312
Validation | D loss=0.416868 | G loss:3.463191 | D(x):0.868181 | D(G(z)) = 0.0313 / 0.0313
Validation | D loss=0.456147 | G loss:3.469190 | D(x):0.853632 | D(G(z)) = 0.0311 / 0.0311
Validation | epoch=003 | D loss=510.839032 | G loss:3913.112793
Train | D loss=0.438477 | G loss:3.169806 | D(x):0.875213 | D(

KeyboardInterrupt: 

In [18]:
torch.save(g.state_dict(), 'Generator_normal_10epochs.pt')